dcd_ch5_hw

# Chapter 5 Homework

## Problem 1
**change Db temp and dialectric perm in travel time analysis**

## Problem 2
**If the temperature of the soil liquid phase increases, does the bulk dielectric permittivity 
increase or decrease? What effect would this change have on soil wate 
content computation?**

## Problem 3
**Compute the matric water potential for water in a saturated pore 10 μm in diameter,
at 20 ◦C. The surface tension and liquid density at this temperature are
72.75 × 10–3 Nm–1 and 1027.8 kgm–3, respectively.**

## Problem 4
**By using the integration presented in the program
PSP_columnWaterContent.py, compute the total amount of soil water
in a clay soil for a profile that is 1m deep. Use the parameters presented in
Table 5.4**

## Problem 5
**By using the two files sand.txt and silt_loam.text, fit Campbell’s equation,
van Genuchten’s equation with no parameter restriction and van Genucthen’s
equation with the restriction m = 1 – 1/n. Print the parameters and discuss the differences
obtained by using the different models. Also discuss the differences in the
estimated parameters for the two soils.**

## Problem 6
**Compute the water content at field capacity (assume ψFC = –33 J kg–1), and
permanent wilting point (assume ψPWP = –1500 J kg–1) using van Genuchten’s
equation for a soil having the following parameters: α = 0.15, n = 1.35, θr = 0.01
and θs = 0.46. The units of water potential are J kg–1. Plant available water is assumed
to be the difference between these values. Compute the plant available water
of the soil.**

## Problem 7
**Derive the capacity for Campbell’s and van Genuchten’s equations by computing
the derivatives of the two functions with respect t psi**
.